In [ ]:
import pandas as pd
data = pd.read_csv('loansTrain.csv')
print(data.shape)
data.head(10)

(1000000, 59)


,loan_amnt,term,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,collections_12_mths_ex_med,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_il_high_credit_limit,loan_status
0,30800,60 months,E,Cardiac cath lab tech,< 1 year,RENT,70000.0,Verified,debt_consolidation,634xx,MO,25.70,1.0,Oct-1999,1.0,6.0,0.0,8343,79.5,0.0,Individual,0.0,0.0,154323.0,10500.0,7.0,25721.0,2157.0,79.5,0.0,0.0,70.0,2.0,2.0,1.0,2.0,2.0,0.0,1.0,1.0,2.0,5.0,8.0,2.0,5.0,1.0,6.0,0.0,0.0,0.0,2.0,92.3,50.0,0.0,0.0,193334.0,55725.0,57234.0,0
1,15000,60 months,D,City of Platte City,3 years,MORTGAGE,80000.0,Source Verified,small_business,640xx,MO,14.04,0.0,Oct-2002,2.0,10.0,0.0,9448,55.3,0.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0
2,3200,36 months,B,NaN,NaN,RENT,30000.0,Not Verified,car,112xx,NY,7.84,0.0,Mar-1999,0.0,4.0,0.0,6982,61.8,0.0,Individual,0.0,0.0,12505.0,11300.0,1.0,3126.0,4318.0,61.8,0.0,0.0,197.0,16.0,16.0,0.0,16.0,NaN,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,16800.0,12505.0,5500.0,1
3,10000,36 months,A,Portfolio Manager,2 years,MORTGAGE,160000.0,Not Verified,credit_card,607xx,IL,7.37,0.0,Jul-2003,1.0,12.0,0.0,24022,77.5,0.0,Individual,0.0,0.0,265667.0,31000.0,2.0,26567.0,6978.0,77.5,0.0,0.0,127.0,47.0,14.0,3.0,47.0,2.0,0.0,3.0,3.0,3.0,7.0,8.0,5.0,10.0,3.0,12.0,0.0,0.0,0.0,0.0,100.0,66.7,0.0,0.0,290467.0,77238.0,55217.0,1
4,34275,60 months,F,Engineer,2 years,RENT,110000.0,Verified,credit_card,112xx,NY,14.86,0.0,Nov-2009,1.0,17.0,0.0,27551,55.5,0.0,Individual,0.0,0.0,43252.0,49600.0,3.0,2544.0,17952.0,59.7,0.0,0.0,87.0,17.0,10.0,0.0,17.0,4.0,0.0,9.0,11.0,13.0,16.0,1.0,16.0,21.0,11.0,17.0,0.0,0.0,0.0,1.0,100.0,30.8,0.0,0.0,69600.0,43252.0,20000.0,1
5,3000,36 months,C,manager,10+ years,MORTGAGE,94199.0,Verified,credit_card,064xx,CT,24.77,1.0,Feb-2002,0.0,13.0,0.0,13610,98.6,0.0,Individual,0.0,245.0,273255.0,13800.0,8.0,21020.0,157.0,98.7,0.0,0.0,160.0,7.0,7.0,4.0,7.0,2.0,0.0,5.0,7.0,5.0,14.0,13.0,7.0,18.0,7.0,13.0,0.0,0.0,0.0,2.0,68.6,100.0,0.0,0.0,314099.0,95364.0,104067.0,1
6,3400,36 months,C,Bellhop,3 years,MORTGAGE,30400.0,Verified,home_improvement,891xx,NV,22.11,0.0,Jan-2014,0.0,9.0,0.0,3913,54.3,0.0,Individual,0.0,242.0,170118.0,7200.0,6.0,18902.0,3287.0,54.3,0.0,0.0,36.0,2.0,2.0,2.0,2.0,5.0,0.0,6.0,6.0,7.0,7.0,4.0,7.0,7.0,6.0,9.0,0.0,0.0,0.0,4.0,100.0,14.3,0.0,0.0,177829.0,23008.0,22229.0,1
7,5000,36 months,A,OFFICE MANAGER,6 years,MORTGAGE,55000.0,Verified,debt_consolidation,933xx,CA,11.29,0.0,Dec-2001,0.0,11.0,0.0,6404,29.8,0.0,Individual,0.0,0.0,155494.0,21500.0,6.0,14136.0,6796.0,48.5,0.0,0.0,186.0,9.0,9.0,1.0,19.0,4.0,3.0,3.0,3.0,4.0,7.0,12.0,9.0,17.0,3.0,11.0,0.0,0.0,0.0,1.0,83.3,25.0,0.0,0.0,179528.0,11162.0,7800.0,1
8,5000,60 months,B,The Kroger Co Fred Meyer Jewelers,10+ years,MORTGAGE,40000.0,Source Verified,debt_consolidation,480xx,MI,14.58,0.0,Dec-1999,2.0,17.0,0.0,7497,12.1,0.0,Individual,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
9,15925,36 months,F,OTR Driver,7 years,OWN,72000.0,Verified,moving,346xx,FL,19.77,1.0,Jan-2000,2.0,6.0,0.0,5912,77.8,0.0,Individual,0.0,0.0,252059.0,7600.0,4.0,42010.0,1688.0,77.8,0.

#Split into 2 parts numeric,categoric.

##Dimention reduce in categorical feature values, drop irrelevant features, dicretization

In [ ]:
import pandas as pd
import numpy as np
#data = pd.read_csv('loansTrain.csv')
data=data.drop(['addr_state', 'emp_title'], axis=1)
conversion_table = {}
conversion_table["home_ownership"] ={"NONE":"MORTGAGE","ANY":"MORTGAGE","OTHER":"MORTGAGE"}
conversion_table["emp_length"] ={"< 1 year":"3>=Year","1 year":"3>=Year","2 years":"3>=Year","3 years":"3>=Year",\
                                 "4 years":"3<=6Year","5 years":"3<=6Year","6 years":"3<=6Year",\
                                 "7 years":">=7Year","8 years":">=7Year","9 years":">=7Year","10+ years":">=7Year"}
total_count = data["purpose"].count()
percent = (data['purpose'].value_counts()/total_count * 100)
purpose_unique = data['purpose'].unique()
purpose_dict = {}
for key,value in percent.items():
  if value <= 2:
    #print(key)
    #data.at[(data["purpose"] == key) , 'purpose'] = "other"
    purpose_dict[key] ="other"
conversion_table["purpose"]= purpose_dict
for category_name,cat_table in conversion_table.items():
  for key,value in cat_table.items():
    data.at[data[category_name] == key, category_name] = value

data["earliest_cr_line"]=pd.to_datetime(data["earliest_cr_line"])
x=data["earliest_cr_line"]
intervals,bins= pd.qcut(x,5,labels=["A","B","C","D","E"],retbins =True)
data["earliest_cr_line"] = intervals
data["earliest_cr_line"]=data["earliest_cr_line"]

def func(s):
  temp = s[0]
  if(temp != "n"):
   return s[0]
  else:
    return "9"
x = data["zip_code"].astype("str")
data["zip_code"]  = x.apply(func,0)

data.at[data["emp_length"].isna(),"emp_length"]= "3<=6Year"

                       

##Handle Test

##Missing value handle

Deleting rows with have too many missing values

In [ ]:
#for i in range(58):
#  print(i,"----",len(data[data.isnull().sum(axis=1) >= i]))
indexes=data[data.isnull().sum(axis=1) >= 10].index
print(indexes)
data=data.drop(indexes,axis=0)
data
data.isnull().sum()


Int64Index([     1,      8,     97,    123,    142,    182,    201,    223,
               230,    232,
            ...
            999835, 999841, 999854, 999864, 999865, 999871, 999875, 999924,
            999935, 999946],
           dtype='int64', length=51675)


loan_amnt                         0
term                              0
grade                             0
emp_length                        0
home_ownership                    0
annual_inc                        0
verification_status               0
purpose                           0
zip_code                          0
dti                             234
delinq_2yrs                       0
earliest_cr_line                  0
inq_last_6mths                    1
open_acc                          0
pub_rec                           0
revol_bal                         0
revol_util                      572
collections_12_mths_ex_med        0
application_type                  0
acc_now_delinq                    0
tot_coll_amt                      0
tot_cur_bal                       0
total_rev_hi_lim                  0
acc_open_past_24mths              0
avg_cur_bal                      18
bc_open_to_buy                10052
bc_util                       10608
chargeoff_within_12_mths    

filling rest of the nan values

In [ ]:
from sklearn.preprocessing import Imputer
numeric_df = data.select_dtypes(include=np.number)
categoric_df=data.select_dtypes(include='object')

In [ ]:
numeric_df

,loan_amnt,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_il_high_credit_limit,loan_status
0,30800,70000.0,25.70,1.0,1.0,6.0,0.0,8343,79.5,0.0,0.0,0.0,154323.0,10500.0,7.0,25721.0,2157.0,79.5,0.0,0.0,70.0,2.0,2.0,1.0,2.0,2.0,0.0,1.0,1.0,2.0,5.0,8.0,2.0,5.0,1.0,6.0,0.0,0.0,0.0,2.0,92.3,50.0,0.0,0.0,193334.0,55725.0,57234.0,0
2,3200,30000.0,7.84,0.0,0.0,4.0,0.0,6982,61.8,0.0,0.0,0.0,12505.0,11300.0,1.0,3126.0,4318.0,61.8,0.0,0.0,197.0,16.0,16.0,0.0,16.0,NaN,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,16800.0,12505.0,5500.0,1
3,10000,160000.0,7.37,0.0,1.0,12.0,0.0,24022,77.5,0.0,0.0,0.0,265667.0,31000.0,2.0,26567.0,6978.0,77.5,0.0,0.0,127.0,47.0,14.0,3.0,47.0,2.0,0.0,3.0,3.0,3.0,7.0,8.0,5.0,10.0,3.0,12.0,0.0,0.0,0.0,0.0,100.0,66.7,0.0,0.0,290467.0,77238.0,55217.0,1
4,34275,110000.0,14.86,0.0,1.0,17.0,0.0,27551,55.5,0.0,0.0,0.0,43252.0,49600.0,3.0,2544.0,17952.0,59.7,0.0,0.0,87.0,17.0,10.0,0.0,17.0,4.0,0.0,9.0,11.0,13.0,16.0,1.0,16.0,21.0,11.0,17.0,0.0,0.0,0.0,1.0,100.0,30.8,0.0,0.0,69600.0,43252.0,20000.0,1
5,3000,94199.0,24.77,1.0,0.0,13.0,0.0,13610,98.6,0.0,0.0,245.0,273255.0,13800.0,8.0,21020.0,157.0,98.7,0.0,0.0,160.0,7.0,7.0,4.0,7.0,2.0,0.0,5.0,7.0,5.0,14.0,13.0,7.0,18.0,7.0,13.0,0.0,0.0,0.0,2.0,68.6,100.0,0.0,0.0,314099.0,95364.0,104067.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,30000,145000.0,27.75,0.0,0.0,12.0,0.0,76491,91.9,0.0,0.0,0.0,190601.0,83200.0,2.0,15883.0,2206.0,97.1,0.0,0.0,361.0,24.0,19.0,2.0,24.0,NaN,0.0,8.0,9.0,8.0,13.0,3.0,9.0,14.0,9.0,12.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,232635.0,100610.0,46235.0,0
999996,6000,80000.0,10.88,1.0,1.0,9.0,3.0,10011,69.0,0.0,0.0,0.0,14761.0,14500.0,0.0,1640.0,1129.0,85.5,0.0,0.0,279.0,25.0,25.0,2.0,39.0,0.0,1.0,3.0,8.0,3.0,14.0,1.0,8.0,22.0,8.0,9.0,0.0,0.0,1.0,0.0,95.8,66.7,0.0,3.0,47249.0,14761.0,32749.0,1
999997,5700,22044.0,25.97,0.0,3.0,11.0,0.0,12522,41.1,0.0,0.0,0.0,75672.0,30500.0,5.0,6879.0,12757.0,45.5,0.0,0.0,231.0,2.0,2.0,3.0,4.0,0.0,0.0,6.0,8.0,6.0,12.0,14.0,9.0,15.0,8.0,11.0,0.0,0.0,0.0,3.0,87.0,33.3,0.0,0.0,101045.0,12987.0,1045.0,1
999998,15000,105000.0,7.53,0.0,0.0,6.0,0.0,15723,44.5,0.0,0.0,0.0,309989.0,35300.0,1.0,51665.0,16931.0,48.1,0.0,0.0,242.0,66.0,24.0,3.0,144.0,1.0,0.0,2.0,3.0,2.0,3.0,2.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,346900.0,27523.0,18000.0,1


In [ ]:
categoric_df

,term,grade,emp_length,home_ownership,verification_status,purpose,zip_code,application_type
0,60 months,E,3>=Year,RENT,Verified,debt_consolidation,6,Individual
2,36 months,B,3<=6Year,RENT,Not Verified,other,1,Individual
3,36 months,A,3>=Year,MORTGAGE,Not Verified,credit_card,6,Individual
4,60 months,F,3>=Year,RENT,Verified,credit_card,1,Individual
5,36 months,C,>=7Year,MORTGAGE,Verified,credit_card,0,Individual
...,...,...,...,...,...,...,...,...
999995,36 months,D,3>=Year,MORTGAGE,Verified,debt_consolidation,7,Individual
999996,36 months,C,3<=6Year,MORTGAGE,Source Verified,other,9,Individual
999997,36 months,B,3<=6Year,MORTGAGE,Verified,debt_consolidation,4,Individual
999998,36 months,A,3>=Year,MORTGAGE,Not Verified,credit_card,2,Individual


In [ ]:
categorical_column_fill=dict()
for i in categoric_df:
  categorical_column_fill[i]=categoric_df[i].value_counts().argmax()
print(categorical_column_fill)

for i in categoric_df:
   categoric_df.at[categoric_df[i].isna(),i] = categorical_column_fill[i]
categoric_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


{'term': ' 36 months', 'grade': 'B', 'emp_length': '>=7Year', 'home_ownership': 'MORTGAGE', 'verification_status': 'Source Verified', 'purpose': 'debt_consolidation', 'zip_code': '9', 'application_type': 'Individual'}


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2909: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[index, col] = value


,term,grade,emp_length,home_ownership,verification_status,purpose,zip_code,application_type
0,60 months,E,3>=Year,RENT,Verified,debt_consolidation,6,Individual
2,36 months,B,3<=6Year,RENT,Not Verified,other,1,Individual
3,36 months,A,3>=Year,MORTGAGE,Not Verified,credit_card,6,Individual
4,60 months,F,3>=Year,RENT,Verified,credit_card,1,Individual
5,36 months,C,>=7Year,MORTGAGE,Verified,credit_card,0,Individual
...,...,...,...,...,...,...,...,...
999995,36 months,D,3>=Year,MORTGAGE,Verified,debt_consolidation,7,Individual
999996,36 months,C,3<=6Year,MORTGAGE,Source Verified,other,9,Individual
999997,36 months,B,3<=6Year,MORTGAGE,Verified,debt_consolidation,4,Individual
999998,36 months,A,3>=Year,MORTGAGE,Not Verified,credit_card,2,Individual


In [ ]:
missing_imp_numeric = Imputer(strategy='median')
columnstofit = numeric_df[numeric_df.columns[:-1]]
missing_imp_numeric.fit(columnstofit)
numeric_df[columnstofit.columns] = missing_imp_numeric.transform(columnstofit)
numeric_df

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,loan_amnt,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_il_high_credit_limit,loan_status
0,30800.0,70000.0,25.70,1.0,1.0,6.0,0.0,8343.0,79.5,0.0,0.0,0.0,154323.0,10500.0,7.0,25721.0,2157.0,79.5,0.0,0.0,70.0,2.0,2.0,1.0,2.0,2.0,0.0,1.0,1.0,2.0,5.0,8.0,2.0,5.0,1.0,6.0,0.0,0.0,0.0,2.0,92.3,50.0,0.0,0.0,193334.0,55725.0,57234.0,0
2,3200.0,30000.0,7.84,0.0,0.0,4.0,0.0,6982.0,61.8,0.0,0.0,0.0,12505.0,11300.0,1.0,3126.0,4318.0,61.8,0.0,0.0,197.0,16.0,16.0,0.0,16.0,5.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,0.0,0.0,0.0,0.0,100.0,50.0,0.0,0.0,16800.0,12505.0,5500.0,1
3,10000.0,160000.0,7.37,0.0,1.0,12.0,0.0,24022.0,77.5,0.0,0.0,0.0,265667.0,31000.0,2.0,26567.0,6978.0,77.5,0.0,0.0,127.0,47.0,14.0,3.0,47.0,2.0,0.0,3.0,3.0,3.0,7.0,8.0,5.0,10.0,3.0,12.0,0.0,0.0,0.0,0.0,100.0,66.7,0.0,0.0,290467.0,77238.0,55217.0,1
4,34275.0,110000.0,14.86,0.0,1.0,17.0,0.0,27551.0,55.5,0.0,0.0,0.0,43252.0,49600.0,3.0,2544.0,17952.0,59.7,0.0,0.0,87.0,17.0,10.0,0.0,17.0,4.0,0.0,9.0,11.0,13.0,16.0,1.0,16.0,21.0,11.0,17.0,0.0,0.0,0.0,1.0,100.0,30.8,0.0,0.0,69600.0,43252.0,20000.0,1
5,3000.0,94199.0,24.77,1.0,0.0,13.0,0.0,13610.0,98.6,0.0,0.0,245.0,273255.0,13800.0,8.0,21020.0,157.0,98.7,0.0,0.0,160.0,7.0,7.0,4.0,7.0,2.0,0.0,5.0,7.0,5.0,14.0,13.0,7.0,18.0,7.0,13.0,0.0,0.0,0.0,2.0,68.6,100.0,0.0,0.0,314099.0,95364.0,104067.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,30000.0,145000.0,27.75,0.0,0.0,12.0,0.0,76491.0,91.9,0.0,0.0,0.0,190601.0,83200.0,2.0,15883.0,2206.0,97.1,0.0,0.0,361.0,24.0,19.0,2.0,24.0,5.0,0.0,8.0,9.0,8.0,13.0,3.0,9.0,14.0,9.0,12.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,232635.0,100610.0,46235.0,0
999996,6000.0,80000.0,10.88,1.0,1.0,9.0,3.0,10011.0,69.0,0.0,0.0,0.0,14761.0,14500.0,0.0,1640.0,1129.0,85.5,0.0,0.0,279.0,25.0,25.0,2.0,39.0,0.0,1.0,3.0,8.0,3.0,14.0,1.0,8.0,22.0,8.0,9.0,0.0,0.0,1.0,0.0,95.8,66.7,0.0,3.0,47249.0,14761.0,32749.0,1
999997,5700.0,22044.0,25.97,0.0,3.0,11.0,0.0,12522.0,41.1,0.0,0.0,0.0,75672.0,30500.0,5.0,6879.0,12757.0,45.5,0.0,0.0,231.0,2.0,2.0,3.0,4.0,0.0,0.0,6.0,8.0,6.0,12.0,14.0,9.0,15.0,8.0,11.0,0.0,0.0,0.0,3.0,87.0,33.3,0.0,0.0,101045.0,12987.0,1045.0,1
999998,15000.0,105000.0,7.53,0.0,0.0,6.0,0.0,15723.0,44.5,0.0,0.0,0.0,309989.0,35300.0,1.0,51665.0,16931.0,48.1,0.0,0.0,242.0,66.0,24.0,3.0,144.0,1.0,0.0,2.0,3.0,2.0,3.0,2.0,4.0,7.0,3.0,6.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,346900.0,27523.0,18000.0,1


In [ ]:
numeric_df.isna().sum()

loan_amnt                     0
annual_inc                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
collections_12_mths_ex_med    0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
total_rev_hi_lim              0
acc_open_past_24mths          0
avg_cur_bal                   0
bc_open_to_buy                0
bc_util                       0
chargeoff_within_12_mths      0
delinq_amnt                   0
mo_sin_old_rev_tl_op          0
mo_sin_rcnt_rev_tl_op         0
mo_sin_rcnt_tl                0
mort_acc                      0
mths_since_recent_bc          0
mths_since_recent_inq         0
num_accts_ever_120_pd         0
num_actv_bc_tl                0
num_actv_rev_tl               0
num_bc_sats                   0
num_bc_tl                     0
num_il_t

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
columnstofit = numeric_df[numeric_df.columns[:-1]]

sc.fit(columnstofit)
numeric_df[columnstofit.columns]= sc.transform(columnstofit)
numeric_df


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,loan_amnt,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_il_high_credit_limit,loan_status
0,1.862541,-0.094617,0.657392,0.754738,0.375463,-1.036407,-0.367245,-0.356820,1.135565,-0.117568,-0.067,-0.028189,0.084817,-0.614197,0.717009,0.752164,-0.521339,0.693659,-0.08502,-0.018655,-1.181865,-0.679901,-0.671226,-0.338995,-0.709389,-0.814140,-0.386144,-1.176700,-1.408407,-0.926592,-0.650643,-0.077475,-1.380403,-1.187807,-1.430419,-1.030685,-0.026438,-0.05484,-0.176621,-0.099244,-0.214730,0.135191,-0.363038,-0.135558,0.108129,0.124279,0.351703,0
2,-1.298810,-0.674253,-0.966615,-0.366470,-0.695123,-1.400169,-0.367245,-0.417389,0.408182,-0.117568,-0.067,-0.028189,-0.818280,-0.592076,-1.162124,-0.635993,-0.379535,0.064077,-0.08502,-0.018655,0.164267,0.179067,0.938843,-0.838962,-0.250815,-0.275983,-0.386144,-0.731689,-1.105426,-0.926592,-1.274676,-0.891487,-1.380403,-1.557124,-1.119309,-1.395492,-0.026438,-0.05484,-0.176621,-1.182885,0.667864,0.135191,-0.363038,-0.135558,-0.887280,-0.782829,-0.848930,1
3,-0.519927,1.209564,-1.009352,-0.366470,0.375463,0.054881,-0.367245,0.340953,1.053374,-0.117568,-0.067,-0.028189,0.793855,-0.047356,-0.848935,0.804139,-0.204987,0.622520,-0.08502,-0.018655,-0.577695,2.081066,0.708833,0.660939,0.764601,-0.814140,-0.386144,-0.286677,-0.802445,-0.588066,-0.234621,-0.077475,-0.721583,-0.572279,-0.808198,0.063735,-0.026438,-0.05484,-0.176621,-1.182885,0.667864,0.601647,-0.363038,-0.135558,0.655826,0.575797,0.304893,1
4,2.260573,0.485019,-0.328288,-0.366470,0.375463,0.964287,-0.367245,0.498006,0.149283,-0.117568,-0.067,-0.028189,-0.622483,0.466948,-0.535746,-0.671749,0.515123,-0.010619,-0.08502,-0.018655,-1.001674,0.240422,0.248813,-0.838962,-0.218059,-0.455369,-0.386144,2.383395,1.621404,2.797197,1.637478,-1.027155,1.694092,0.781882,1.680684,0.975752,-0.026438,-0.05484,-0.176621,-0.641065,0.667864,-0.401094,-0.363038,-0.135558,-0.589561,-0.137507,-0.512417,1
5,-1.321719,0.256048,0.572827,0.754738,-0.695123,0.236762,-0.367245,-0.122419,1.920480,-0.117568,-0.067,0.000063,0.842175,-0.522949,1.030198,0.463351,-0.652578,1.376595,-0.08502,-0.018655,-0.227913,-0.373126,-0.096201,1.160906,-0.545613,-0.814140,-0.386144,0.603347,0.409479,0.088987,1.221456,0.600868,-0.282369,0.412565,0.436243,0.246138,-0.026438,-0.05484,-0.176621,-0.099244,-2.931286,1.531766,-0.363038,-0.135558,0.789078,0.956228,1.438595,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1.770907,0.992201,0.843798,-0.366470,-0.695123,0.054881,-0.367245,2.676014,1.645143,-0.117568,-0.067,-0.028189,0.315835,1.396013,-0.848935,0.147752,-0.518124,1.319684,-0.08502,-0.018655,1.902579,0.669905,1.283858,0.160972,0.011228,-0.275983,-0.386144,1.938383,1.015441,1.104566,1.013445,-0.755818,0.156844,-0.079857,1.058463,0.063735,-0.026438,-0.05484,-0.176621,-1.182885,0.667864,1.531766,-0.363038,-0.135558,0.329733,1.066332,0.096440,0
999996,-0.978094,0.050292,-0.690188,0.754738,0.375463,-0.490763,4.536002,-0.282588,0.704066,-0.117568,-0.067,-0.028189,-0.803913,-0.503594,-1.475313,-0.727288,-0.588796,0.907077,-0.08502,-0.018655,1.033423,0.731260,1.973887,0.160972,0.502558,-1.172912,0.370556,-0.286677,0.712460,-0.588066,1.221456,-1.027155,-0.062762,0.904987,0.747353,-0.483475,-0.026438,-0.05484,1.

## PCA analysis

In [ ]:
counts = categoric_df.nunique()
categorical_columns =categoric_df.columns
binaries = []
multi_categorical_columns = []
for i in range(len(counts)):
  if counts[i] == 2:
    binaries.append(categorical_columns[i])
print("Binaries: ",binaries)
binary_df = categoric_df[binaries]
for i in categorical_columns:
  if i not in binaries:
    multi_categorical_columns.append(i)
print("MultiCategoricals: ", multi_categorical_columns)
multi_col_df = categoric_df[multi_categorical_columns]


Binaries:  ['term', 'application_type']
MultiCategoricals:  ['grade', 'emp_length', 'home_ownership', 'verification_status', 'purpose', 'zip_code']


In [ ]:
binary_df

,term,application_type
0,60 months,Individual
2,36 months,Individual
3,36 months,Individual
4,60 months,Individual
5,36 months,Individual
...,...,...
999995,36 months,Individual
999996,36 months,Individual
999997,36 months,Individual
999998,36 months,Individual


In [ ]:
multi_col_df

,grade,emp_length,home_ownership,verification_status,purpose,zip_code
0,E,3>=Year,RENT,Verified,debt_consolidation,6
2,B,3<=6Year,RENT,Not Verified,other,1
3,A,3>=Year,MORTGAGE,Not Verified,credit_card,6
4,F,3>=Year,RENT,Verified,credit_card,1
5,C,>=7Year,MORTGAGE,Verified,credit_card,0
...,...,...,...,...,...,...
999995,D,3>=Year,MORTGAGE,Verified,debt_consolidation,7
999996,C,3<=6Year,MORTGAGE,Source Verified,other,9
999997,B,3<=6Year,MORTGAGE,Verified,debt_consolidation,4
999998,A,3>=Year,MORTGAGE,Not Verified,credit_card,2


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

term_binary_encoder = LabelEncoder()
term_binary_encoder.fit(binary_df["term"])
binary_df["term"]=term_binary_encoder.transform(binary_df["term"]).tolist()


application_type_encoder =LabelEncoder()
application_type_encoder.fit(binary_df["application_type"])
binary_df["application_type"]=application_type_encoder.transform(binary_df["application_type"]).tolist()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
binary_df

,term,application_type
0,1,0
2,0,0
3,0,0
4,1,0
5,0,0
...,...,...
999995,0,0
999996,0,0
999997,0,0
999998,0,0


In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
Enc_label =LabelEncoder()

other_encoders = OneHotEncoder(handle_unknown='ignore')
other_encoders.fit(multi_col_df)

a=pd.get_dummies(multi_col_df)
a_=a.values.tolist()

multi_col_df=pd.DataFrame(other_encoders.transform(multi_col_df).todense().tolist(),columns = a.columns,index = multi_col_df.index)
multi_col_df

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,emp_length_3<=6Year,emp_length_3>=Year,emp_length_>=7Year,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_major_purchase,purpose_other,zip_code_0,zip_code_1,zip_code_2,zip_code_3,zip_code_4,zip_code_5,zip_code_6,zip_code_7,zip_code_8,zip_code_9
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
999996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
999997,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
999998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
numeric_df.head(5)

,loan_amnt,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_il_high_credit_limit,loan_status
0,1.862541,-0.094617,0.657392,0.754738,0.375463,-1.036407,-0.367245,-0.356820,1.135565,-0.117568,-0.067,-0.028189,0.084817,-0.614197,0.717009,0.752164,-0.521339,0.693659,-0.08502,-0.018655,-1.181865,-0.679901,-0.671226,-0.338995,-0.709389,-0.814140,-0.386144,-1.176700,-1.408407,-0.926592,-0.650643,-0.077475,-1.380403,-1.187807,-1.430419,-1.030685,-0.026438,-0.05484,-0.176621,-0.099244,-0.214730,0.135191,-0.363038,-0.135558,0.108129,0.124279,0.351703,0
2,-1.298810,-0.674253,-0.966615,-0.366470,-0.695123,-1.400169,-0.367245,-0.417389,0.408182,-0.117568,-0.067,-0.028189,-0.818280,-0.592076,-1.162124,-0.635993,-0.379535,0.064077,-0.08502,-0.018655,0.164267,0.179067,0.938843,-0.838962,-0.250815,-0.275983,-0.386144,-0.731689,-1.105426,-0.926592,-1.274676,-0.891487,-1.380403,-1.557124,-1.119309,-1.395492,-0.026438,-0.05484,-0.176621,-1.182885,0.667864,0.135191,-0.363038,-0.135558,-0.887280,-0.782829,-0.848930,1
3,-0.519927,1.209564,-1.009352,-0.366470,0.375463,0.054881,-0.367245,0.340953,1.053374,-0.117568,-0.067,-0.028189,0.793855,-0.047356,-0.848935,0.804139,-0.204987,0.622520,-0.08502,-0.018655,-0.577695,2.081066,0.708833,0.660939,0.764601,-0.814140,-0.386144,-0.286677,-0.802445,-0.588066,-0.234621,-0.077475,-0.721583,-0.572279,-0.808198,0.063735,-0.026438,-0.05484,-0.176621,-1.182885,0.667864,0.601647,-0.363038,-0.135558,0.655826,0.575797,0.304893,1
4,2.260573,0.485019,-0.328288,-0.366470,0.375463,0.964287,-0.367245,0.498006,0.149283,-0.117568,-0.067,-0.028189,-0.622483,0.466948,-0.535746,-0.671749,0.515123,-0.010619,-0.08502,-0.018655,-1.001674,0.240422,0.248813,-0.838962,-0.218059,-0.455369,-0.386144,2.383395,1.621404,2.797197,1.637478,-1.027155,1.694092,0.781882,1.680684,0.975752,-0.026438,-0.05484,-0.176621,-0.641065,0.667864,-0.401094,-0.363038,-0.135558,-0.589561,-0.137507,-0.512417,1
5,-1.321719,0.256048,0.572827,0.754738,-0.695123,0.236762,-0.367245,-0.122419,1.920480,-0.117568,-0.067,0.000063,0.842175,-0.522949,1.030198,0.463351,-0.652578,1.376595,-0.08502,-0.018655,-0.227913,-0.373126,-0.096201,1.160906,-0.545613,-0.814140,-0.386144,0.603347,0.409479,0.088987,1.221456,0.600868,-0.282369,0.412565,0.436243,0.246138,-0.026438,-0.05484,-0.176621,-0.099244,-2.931286,1.531766,-0.363038,-0.135558,0.789078,0.956228,1.438595,1


In [ ]:
binary_df.head(5)

,term,application_type
0,1,0
2,0,0
3,0,0
4,1,0
5,0,0


In [ ]:
multi_col_df.head(5)

,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,emp_length_3<=6Year,emp_length_3>=Year,emp_length_>=7Year,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_major_purchase,purpose_other,zip_code_0,zip_code_1,zip_code_2,zip_code_3,zip_code_4,zip_code_5,zip_code_6,zip_code_7,zip_code_8,zip_code_9
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
categoric_columns = binary_df.columns.tolist() + multi_col_df.columns.tolist()
df=pd.concat([numeric_df,binary_df,multi_col_df],axis=1)
categoric_columns

['term',
 'application_type',
 'grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'emp_length_3<=6Year',
 'emp_length_3>=Year',
 'emp_length_>=7Year',
 'home_ownership_MORTGAGE',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'verification_status_Not Verified',
 'verification_status_Source Verified',
 'verification_status_Verified',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_home_improvement',
 'purpose_major_purchase',
 'purpose_other',
 'zip_code_0',
 'zip_code_1',
 'zip_code_2',
 'zip_code_3',
 'zip_code_4',
 'zip_code_5',
 'zip_code_6',
 'zip_code_7',
 'zip_code_8',
 'zip_code_9']

In [ ]:
df.head()


,loan_amnt,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,collections_12_mths_ex_med,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_il_high_credit_limit,loan_status,term,application_type,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,emp_length_3<=6Year,emp_length_3>=Year,emp_length_>=7Year,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_major_purchase,purpose_other,zip_code_0,zip_code_1,zip_code_2,zip_code_3,zip_code_4,zip_code_5,zip_code_6,zip_code_7,zip_code_8,zip_code_9
0,1.862541,-0.094617,0.657392,0.754738,0.375463,-1.036407,-0.367245,-0.356820,1.135565,-0.117568,-0.067,-0.028189,0.084817,-0.614197,0.717009,0.752164,-0.521339,0.693659,-0.08502,-0.018655,-1.181865,-0.679901,-0.671226,-0.338995,-0.709389,-0.814140,-0.386144,-1.176700,-1.408407,-0.926592,-0.650643,-0.077475,-1.380403,-1.187807,-1.430419,-1.030685,-0.026438,-0.05484,-0.176621,-0.099244,...,0.135191,-0.363038,-0.135558,0.108129,0.124279,0.351703,0,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-1.298810,-0.674253,-0.966615,-0.366470,-0.695123,-1.400169,-0.367245,-0.417389,0.408182,-0.117568,-0.067,-0.028189,-0.818280,-0.592076,-1.162124,-0.635993,-0.379535,0.064077,-0.08502,-0.018655,0.164267,0.179067,0.938843,-0.838962,-0.250815,-0.275983,-0.386144,-0.731689,-1.105426,-0.926592,-1.274676,-0.891487,-1.380403,-1.557124,-1.119309,-1.395492,-0.026438,-0.05484,-0.176621,-1.182885,...,0.135191,-0.363038,-0.135558,-0.887280,-0.782829,-0.848930,1,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.519927,1.209564,-1.009352,-0.366470,0.375463,0.054881,-0.367245,0.340953,1.053374,-0.117568,-0.067,-0.028189,0.793855,-0.047356,-0.848935,0.804139,-0.204987,0.622520,-0.08502,-0.018655,-0.577695,2.081066,0.708833,0.660939,0.764601,-0.814140,-0.386144,-0.286677,-0.802445,-0.588066,-0.234621,-0.077475,-0.721583,-0.572279,-0.808198,0.063735,-0.026438,-0.05484,-0.176621,-1.182885,...,0.601647,-0.363038,-0.135558,0.655826,0.575797,0.304893,1,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2.260573,0.485019,-0.328288,-0.366470,0.375463,0.964287,-0.367245,0.498006,0.149283,-0.117568,-0.067,-0.028189,-0.622483,0.466948,-0.535746,-0.671749,0.515123,-0.010619,-0.08502,-0.018655,-1.001674,0.240422,0.248813,-0.838962,-0.218059,-0.455369,-0.386144,2.383395,1.621404,2.797197,1.637478,-1.027155,1.694092,0.781882,1.680684,0.975752,-0.026438,-0.05484,-0.176621,-0.641065,...,-0.401094,-0.363038,-0.135558,-0.589561,-0.137507,-0.512417,1,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-1.321719,0.256048,0.572827,0.754738,-0.695123,0.236762,-0.367245,-0.122419,1.920480,-0.117568,-0.067,0.000063,0.842175,-0.522949,1.030198,0.463351,-0.652578,1.376595,-0.08502,-0.018655,-0.227913,-0.373126,-0.096201,1.160906,-0.545613,-0.814140,-0.386144,0.603347,0.409479,0.088987,1.221456,0.600868,-0.282369,0.412565,0.436243,0.246138,-0.026438,-0.05484,-0.176621,-0.099244,...,1.531766,-0.363038,-0.135558,0.789078,0.956228,1.438595,1,0,0,0.0,0.0,1.0,0.0,0

In [ ]:
df.isna().sum().sum()

0

## Sentetic data sythesis using knn - smote

In [ ]:
y = df['loan_status']
X=df[[i for i in df.columns if i != "loan_status"]]
print("Class Disturbition", y.value_counts())

Class Disturbition 1    755611
0    192714
Name: loan_status, dtype: int64


In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy="auto")
X_res, y_res = sm.fit_resample(X, y)


In [ ]:
from imblearn.over_sampling import SMOTENC
q = []
for i in X.columns:
  if i in categoric_columns:
    q.append(True)
  else:
    q.append(False)
smote_nc = SMOTENC(sampling_strategy=0.5,categorical_features=q, random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X, y)


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


KeyboardInterrupt: ignored

# Split into target column and features


In [ ]:

X.head()

# Shuffle and split the data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
####### Statis.

def ResultsOfModel(test_pred,y_test):
  true_positive_m = (test_pred == 1) & (y_test == 1)
  true_positive = len(test_pred[true_positive_m])
  print('True Positive = ' + str(true_positive))

  false_positive_m = (test_pred == 1) & (y_test == 0)
  false_positive = len(test_pred[false_positive_m])
  print('False Positive = ' + str(false_positive))

  true_negative_m = (test_pred == 0) & (y_test == 0)
  true_negative = len(test_pred[true_negative_m])
  print('True Negative = ' + str(true_negative))

  false_negative_m = (test_pred == 0) & (y_test == 1)
  false_negative = len(test_pred[false_negative_m])
  print('False Negative = ' + str(false_negative))

  precision = (true_negative)/(true_negative + false_negative)
  recall    =  (true_negative) / (true_negative + false_positive)
  f1_score  = 2* (precision * recall) / (precision + recall)
  #mcc = (true_positive * true_negative - false_positive * false_negative) / np.sqrt((true_positive + false_positive) * (true_positive + false_negative) * (true_negative + false_positive) * (true_negative + false_negative))

  print('Precision = ' + str(precision))
  print('Recall = ' + str(recall))
  print('F1 Score = ' + str(f1_score))
 # print('mcc = ' + str(mcc))


# Apply Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_Logistic = LogisticRegression(random_state=0).fit(X_train, y_train)
train_pred= clf_Logistic.predict(X_train)
test_pred = clf_Logistic.predict(X_test)

In [ ]:
print("Train Error")
ResultsOfModel(train_pred,y_train)
print("___________________")
print("")
print("Validation Error")
ResultsOfModel(test_pred,y_test)

##Apply Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf_randomForest = RandomForestClassifier(min_samples_split=20 ,random_state=0)
clf_randomForest.fit(X_train,y_train)
train_pred= clf_randomForest.predict(X_train)
test_pred = clf_randomForest.predict(X_test)



In [ ]:
important_features_dict = {}
for x,i in enumerate(clf_randomForest.feature_importances_):
    important_features_dict[df.columns[x]]=i


important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)
print(important_features_list)
values_list = [important_features_dict[i] for i in important_features_list]
print(values_list)

In [ ]:
import matplotlib.pyplot as plt

# Rearrange feature names so they match the sorted feature importances
names = X.columns

# Create plot
plt.figure(figsize=(40,15))

# Create plot title
plt.title("Feature Importance")

# Add bars
plt.bar(range(X.shape[1]), values_list)

# Add feature names as x-axis labels
plt.xticks(range(X.shape[1]), important_features_list, rotation=80)

# Show plot
plt.show()

In [ ]:

print("Train Error")
ResultsOfModel(train_pred,y_train)
print("___________________")
print("")
print("Validation Error")
ResultsOfModel(test_pred,y_test)

## Erasing Irrelevant Features

In [ ]:
df.drop()

#Hyperparameter Optimization for Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

for i in {10,20,50}:
    clf_randomForest = RandomForestClassifier(min_samples_split=i ,random_state=0)
    clf_randomForest.fit(X_train,y_train)
    train_pred= clf_randomForest.predict(X_train)
    test_pred = clf_randomForest.predict(X_test)

    print("Train Error for", i)
    ResultsOfModel(train_pred,y_train)
    print("___________________")
    print("")
    print("Validation Error for", i)
    ResultsOfModel(test_pred,y_test)
    print("")
    print("")
    print("")

##Apply AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

clf_AdaBoost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                                  n_estimators=50,
                                  learning_rate=1.0,
                                  algorithm="SAMME.R",
                                  random_state=None)

clf_AdaBoost.fit(X_train, y_train)
test_pred = clf_AdaBoost.predict(X_test)

In [ ]:
ResultsOfModel(test_pred,y_test)

#Apply on test

In [ ]:
test_data = pd.read_csv('loansTest.csv',index_col="ID")
test_data=test_data.drop(['addr_state', 'emp_title',"zip_code"], axis=1)

for category_name,cat_table in conversion_table.items():
  for key,value in cat_table.items():
    test_data.at[test_data[category_name] == key, category_name] = value

test_data.at[test_data["emp_length"].isna(),"emp_length"]= "3<=6Year"

test_data["earliest_cr_line"]=pd.to_datetime(test_data["earliest_cr_line"])

intervals= pd.cut(test_data["earliest_cr_line"],bins,labels=["A","B","C","D","E"])
test_data["earliest_cr_line"] = intervals



test_numeric_df = test_data.select_dtypes(include=np.number)
test_categoric_df=test_data.select_dtypes(include='object')


for i in test_categoric_df:
   test_categoric_df.at[test_categoric_df[i].isna(),i] = categorical_column_fill[i]

test_numeric_df[test_numeric_df.columns] = missing_imp_numeric.transform(test_numeric_df)
test_numeric_df[test_numeric_df.columns]= sc.transform(test_numeric_df)


test_binary_df = test_categoric_df[binaries]
test_multi_col_df = test_categoric_df[multi_categorical_columns]


test_binary_df["term"]=term_binary_encoder.transform(test_binary_df["term"]).tolist()
test_binary_df["application_type"]=application_type_encoder.transform(test_binary_df["application_type"]).tolist()

test_multi_col_df=pd.DataFrame(other_encoders.transform(test_multi_col_df).todense().tolist(),columns = a.columns,index = test_multi_col_df.index)

print(test_binary_df.isna().sum())
print(test_numeric_df.isna().sum())
print(test_multi_col_df.isna().sum())
X_test=pd.concat([test_numeric_df,test_binary_df,test_multi_col_df],axis=1)
X_test

In [ ]:
test_pred = clf.predict(X_test)
pd.DataFrame(test_pred, index=test_data.index)

In [ ]:
temp_dict= {"ID":test_data.index,"loan_status":test_pred.tolist()}

pd.DataFrame(temp_dict).to_csv("cs412result.csv",index=False)
